In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch, requests

# Fetch Dataset
def fetch_dataset(url, filename="dataset.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(requests.get(url).text)
    return filename

dataset_file = fetch_dataset("https://www.gutenberg.org/files/36/36-0.txt")

In [ ]:
# Load & Tokenize Dataset
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
dataset = load_dataset("text", data_files={"train": dataset_file})["train"].train_test_split(test_size=0.1)
dataset = dataset.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=128),
                      batched=True, remove_columns=["text"])

# Load Model & Training
model = GPT2LMHeadModel.from_pretrained("gpt2")
trainer = Trainer(
    model=model,
    args=TrainingArguments(output_dir="./results", eval_strategy="epoch", learning_rate=5e-5,
                           per_device_train_batch_size=4, per_device_eval_batch_size=4, num_train_epochs=3,
                           weight_decay=0.01, save_strategy="epoch", logging_dir="./logs", fp16=torch.cuda.is_available(),
                           report_to="none"),
    train_dataset=dataset["train"], eval_dataset=dataset["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6073 [00:00<?, ? examples/s]

Map:   0%|          | 0/675 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,3.834700,nan
2,3.193000,nan
3,2.898000,nan


TrainOutput(global_step=4557, training_loss=3.3484906717065606, metrics={'train_runtime': 604.4128, 'train_samples_per_second': 30.143, 'train_steps_per_second': 7.54, 'total_flos': 1190119882752000.0, 'train_loss': 3.3484906717065606, 'epoch': 3.0})

In [ ]:
# AI Storytelling Chatbot
def generate_text(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(output[0], skip_special_tokens=True)

print("Welcome to AI Storytelling! Type a prompt or 'exit' to stop.")
while (user_input := input("You: ")) != "exit":
    print("AI:", generate_text(user_input))


Welcome to AI Storytelling! Type a prompt or 'exit' to stop.
You: astronaut in space riding a horse
AI: astronaut in space riding a horse. He was a man of considerable intelligence, and I found him quite interested in the affairs of the people in the kitchen. He was a man of extraordinary intelligence, and I found him quite interested in the affairs of the people in the kitchen. He was a man of extraordinary intelligence, and I found him quite interested in the affairs of the people in the kitchen. He was a man of remarkable intelligence, and I found him quite interested in the affairs of the
You: bike flying
AI: bike flying over the trees, and the houses in the lane were ransacked. The people in the houses were shouting and shouting, and the horses and carts were being driven out. The people in the houses were shouting and shouting, and the horses and carts were being driven out. The people in the houses were shouting and shouting, and the horses and carts were being driven out. The p